In [18]:
import os
import pandas as pd
import numpy as np
import librosa
from os.path import join, normpath
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif

### Requirement Gathering

In [18]:
# All Fixed Variabels
RANDOM_STATE = 26
TEST_SIZE = 0.30

MIX_CSV_PATH = "../data/HLS-CMDS/Mix.csv"
MIX_AUDIO_PATH = "../data/HLS-CMDS/Mix/Mix"
LS_CSV_PATH = "../data/HLS-CMDS/LS.csv"
LS_AUDIO_PATH = "../data/HLS-CMDS/LS/LS"

## Mix Data

##### Gender

In [24]:
df_gender_mix = pd.read_csv(MIX_CSV_PATH)
N_MFCC_GENDER_MIX = 45

df_gender_mix['audio_path'] = df_gender_mix['Mixed Sound ID'].apply(lambda x: normpath(join(MIX_AUDIO_PATH, f"{str(x).strip()}.wav").replace('\\', '/')))

def extract_mfcc_gender_mix(filename, n_mfcc=N_MFCC_GENDER_MIX):
    y, sr = librosa.load(filename, sr=44100)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    return np.concatenate([mfcc_mean, mfcc_std])

# def flatten_mfcc(mfcc):
#     return np.array(mfcc).flatten()

mfcc_list_gender_mix = []
labels_gender_mix = []

for _, row in df_gender_mix.iterrows():
    try:
        mfcc_gender_mix = extract_mfcc_gender_mix(row['audio_path'])
        # mfcc_flat = flatten_mfcc(mfcc)
        mfcc_list_gender_mix.append(mfcc_gender_mix)
        labels_gender_mix.append(row['Gender'])
    except Exception as e:
        print(f"[WARN] Failed: {row['audio_path']} -> {e}")
    
X_gender_mix = pd.DataFrame(mfcc_list_gender_mix)
y_gender_mix = pd.Series(labels_gender_mix)

Xtrain_gender_mix, Xtest_gender_mix, ytrain_gender_mix, ytest_gender_mix = train_test_split(X_gender_mix, y_gender_mix, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_gender_mix)

pipe_model_gender_mix = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', random_state=RANDOM_STATE, class_weight='balanced'))
])

param_grid_gender_mix = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['rbf', 'linear', 'poly']
}

model_grid_gender_mix = GridSearchCV(
    estimator=pipe_model_gender_mix, 
    param_grid=param_grid_gender_mix, 
    scoring='f1_macro', 
    cv=3, 
    n_jobs=-1, 
    verbose=2
)

model_grid_gender_mix.fit(Xtrain_gender_mix, ytrain_gender_mix)
model_gender_mix = model_grid_gender_mix.best_estimator_
print(f"Best parameters: {model_grid_gender_mix.best_params_}")

ypred_gender_mix = model_gender_mix.predict(Xtest_gender_mix)
f1_gender_mix = f1_score(ytest_gender_mix, ypred_gender_mix, average='macro')
print(f"Mix: Gender -> macro-F1: {f1_gender_mix}")

print("Classification Report:")
print(classification_report(ytest_gender_mix, ypred_gender_mix))
print("Confusion Matrix:")
print(confusion_matrix(ytest_gender_mix, ypred_gender_mix))

Fitting 3 folds for each of 48 candidates, totalling 144 fits
Best parameters: {'svc__C': 1, 'svc__gamma': 0.001, 'svc__kernel': 'linear'}
Mix: Gender -> macro-F1: 0.5445134575569358
Classification Report:
              precision    recall  f1-score   support

           F       0.54      0.59      0.57        22
           M       0.55      0.50      0.52        22

    accuracy                           0.55        44
   macro avg       0.55      0.55      0.54        44
weighted avg       0.55      0.55      0.54        44

Confusion Matrix:
[[13  9]
 [11 11]]


##### Heart Sound Type

In [ ]:
N_MFCC_HEARTSOUNDTYPE = 65
df_heartsoundtype_mix = pd.read_csv(MIX_CSV_PATH)

df_heartsoundtype_mix['audio_path'] = df_heartsoundtype_mix['Mixed Sound ID'].apply(lambda x: normpath(join(MIX_AUDIO_PATH, f"{str(x).strip()}.wav").replace('\\', '/')))

def extract_mfcc(filename, n_mfcc=N_MFCC_HEARTSOUNDTYPE):
    y, sr = librosa.load(filename, sr=2000)  # ↓ better for heart sounds
    
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)

    features = np.concatenate([
        np.mean(mfcc, axis=1),
        np.std(mfcc, axis=1),
        np.mean(delta, axis=1),
        np.std(delta, axis=1),
        np.mean(delta2, axis=1),
        np.std(delta2, axis=1)
    ])

    return features

def flatten_mfcc(mfcc):
    return np.array(mfcc).flatten()

mfcc_list_heartsoundtype_mix = []
labels_heartsoundtype_mix = []

for _, row in df_heartsoundtype_mix.iterrows():
    try:
        mfcc_heartsoundtype = extract_mfcc(row['audio_path'])
        # mfcc_flat = flatten_mfcc(mfcc)
        mfcc_list_heartsoundtype_mix.append(mfcc_heartsoundtype)
        labels_heartsoundtype_mix.append(row['Heart Sound Type'])
    except Exception as e:
        print(f"[WARN] Failed: {row['audio_path']} -> {e}")

X_heartsoundtype_mix = pd.DataFrame(mfcc_list_heartsoundtype_mix)
y_heartsoundtype_mix = pd.Series(labels_heartsoundtype_mix)
Xtrain_heartsoundtype_mix, Xtest_heartsoundtype_mix, ytrain_heartsoundtype_mix, ytest_heartsoundtype_mix = train_test_split(X_heartsoundtype_mix, y_heartsoundtype_mix, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_heartsoundtype_mix)

pipe_model_heartsoundtype_mix = Pipeline(steps=[
    ('scaler', StandardScaler()), 
    ('svc', SVC(kernel='rbf', random_state=RANDOM_STATE, class_weight='balanced'))
])

param_grid_heartsoundtype_mix = {
    'svc__C': [0.1, 1, 10, 50, 100],
    'svc__gamma': [0.001, 0.01, 0.05, 0.1, 1],
    'svc__kernel': ['rbf', 'linear', 'poly']
}

model_grid_heartsoundtype_mix = GridSearchCV(
    estimator=pipe_model_heartsoundtype_mix,
    param_grid=param_grid_heartsoundtype_mix,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=2
)

model_grid_heartsoundtype_mix.fit(Xtrain_heartsoundtype_mix, ytrain_heartsoundtype_mix)
model_heartsoundtype_mix = model_grid_heartsoundtype_mix.best_estimator_
print(f"Best parameters: {model_grid_heartsoundtype_mix.best_params_}")
ypred_heartsoundtype_mix = model_heartsoundtype_mix.predict(Xtest_heartsoundtype_mix)
f1_heartsoundtype_mix = f1_score(ytest_heartsoundtype_mix, ypred_heartsoundtype_mix, average='macro')
print(f"Mix: Heart Sound Type -> macro-F1: {f1_heartsoundtype_mix}")
print("Classification Report:")
print(classification_report(ytest_heartsoundtype_mix, ypred_heartsoundtype_mix))
print("Confusion Matrix:")
print(confusion_matrix(ytest_heartsoundtype_mix, ypred_heartsoundtype_mix))

Fitting 5 folds for each of 75 candidates, totalling 375 fits
Best parameters: {'svc__C': 0.1, 'svc__gamma': 0.001, 'svc__kernel': 'linear'}
Mix: Heart Sound Type -> macro-F1: 0.537100122100122
Classification Report:
                       precision    recall  f1-score   support

             AV Block       0.67      0.50      0.57         4
  Atrial Fibrillation       0.00      0.00      0.00         4
Early Systolic Murmur       0.50      1.00      0.67         4
Late Diastolic Murmur       0.75      0.75      0.75         4
 Late Systolic Murmur       0.62      1.00      0.77         5
  Mid Systolic Murmur       0.50      0.25      0.33         4
               Normal       1.00      0.25      0.40         4
                   S3       1.00      0.80      0.89         5
                   S4       0.62      1.00      0.77         5
          Tachycardia       0.25      0.20      0.22         5

             accuracy                           0.59        44
            macro avg    

##### Lung Sound Type

In [29]:
N_MFCC_LUNGSOUNDTYPE_MIX = 60
df_lungsoundtype_mix = pd.read_csv(MIX_CSV_PATH)
df_lungsoundtype_mix['audio_path'] = df_lungsoundtype_mix['Mixed Sound ID'].apply(lambda x: normpath(join(MIX_AUDIO_PATH, f"{str(x).strip()}.wav").replace('\\', '/')))
def extract_mfcc_lungsoundtype_mix(filename, n_mfcc=N_MFCC_LUNGSOUNDTYPE_MIX):
    y, sr = librosa.load(filename, sr=44100)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    return np.concatenate([mfcc_mean, mfcc_std])

mfcc_list_lungsoundtype_mix = []
labels_lungsoundtype_mix = []

for _, row in df_lungsoundtype_mix.iterrows():
    try:
        mfcc_lungsoundtype_mix = extract_mfcc_lungsoundtype_mix(row['audio_path'])
        # mfcc_flat = flatten_mfcc(mfcc)
        mfcc_list_lungsoundtype_mix.append(mfcc_lungsoundtype_mix)
        labels_lungsoundtype_mix.append(row['Lung Sound Type'])
    except Exception as e:
        print(f"[WARN] Failed: {row['audio_path']} -> {e}")

X_lungsoundtype_mix = pd.DataFrame(mfcc_list_lungsoundtype_mix)
y_lungsoundtype_mix = pd.Series(labels_lungsoundtype_mix)

Xtrain_lungsoundtype_mix, Xtest_lungsoundtype_mix, ytrain_lungsoundtype_mix, ytest_lungsoundtype_mix = train_test_split(X_lungsoundtype_mix, y_lungsoundtype_mix, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_lungsoundtype_mix)
pipe_model_lungsoundtype_mix = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', random_state=RANDOM_STATE, class_weight='balanced'))
])
param_grid_lungsoundtype_mix = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['rbf', 'linear', 'poly']
}
model_grid_lungsoundtype_mix = GridSearchCV(
    estimator=pipe_model_lungsoundtype_mix,
    param_grid=param_grid_lungsoundtype_mix,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=2
)
model_grid_lungsoundtype_mix.fit(Xtrain_lungsoundtype_mix, ytrain_lungsoundtype_mix)
model_lungsoundtype_mix = model_grid_lungsoundtype_mix.best_estimator_
print(f"Best parameters: {model_grid_lungsoundtype_mix.best_params_}")
ypred_lungsoundtype_mix = model_lungsoundtype_mix.predict(Xtest_lungsoundtype_mix)
f1_lungsoundtype_mix = f1_score(ytest_lungsoundtype_mix, ypred_lungsoundtype_mix, average='macro')
print(f"Mix: Lung Sound Type -> macro-F1: {f1_lungsoundtype_mix}")
print("Classification Report:")
print(classification_report(ytest_lungsoundtype_mix, ypred_lungsoundtype_mix))
print("Confusion Matrix:")
print(confusion_matrix(ytest_lungsoundtype_mix, ypred_lungsoundtype_mix))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'svc__C': 10, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
Mix: Lung Sound Type -> macro-F1: 0.8361416361416363
Classification Report:
                 precision    recall  f1-score   support

Coarse Crackles       0.71      0.83      0.77         6
  Fine Crackles       0.75      0.86      0.80         7
         Normal       0.83      0.62      0.71         8
    Pleural Rub       0.86      0.75      0.80         8
        Rhonchi       0.88      1.00      0.93         7
       Wheezing       1.00      1.00      1.00         8

       accuracy                           0.84        44
      macro avg       0.84      0.84      0.84        44
   weighted avg       0.85      0.84      0.84        44

Confusion Matrix:
[[5 1 0 0 0 0]
 [1 6 0 0 0 0]
 [1 0 5 1 1 0]
 [0 1 1 6 0 0]
 [0 0 0 0 7 0]
 [0 0 0 0 0 8]]


##### Location

In [22]:
N_MFCC_LOCATION_MIX = 2
df_location_mix = pd.read_csv(MIX_CSV_PATH)
df_location_mix['audio_path'] = df_location_mix['Mixed Sound ID'].apply(lambda x: normpath(join(MIX_AUDIO_PATH, f"{str(x).strip()}.wav").replace('\\', '/')))
def extract_mfcc_location_mix(filename, n_mfcc=N_MFCC_LOCATION_MIX):
    y, sr = librosa.load(filename, sr=166)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    return np.concatenate([mfcc_mean, mfcc_std])
mfcc_list_location_mix = []
labels_location_mix = []

for _, row in df_location_mix.iterrows():
    try:
        mfcc_location_mix = extract_mfcc_location_mix(row['audio_path'])
        # mfcc_flat = flatten_mfcc(mfcc)
        mfcc_list_location_mix.append(mfcc_location_mix)
        labels_location_mix.append(row['Location'])
    except Exception as e:
        print(f"[WARN] Failed: {row['audio_path']} -> {e}")

X_location_mix = pd.DataFrame(mfcc_list_location_mix)
y_location_mix = pd.Series(labels_location_mix)

Xtrain_location_mix, Xtest_location_mix, ytrain_location_mix, ytest_location_mix = train_test_split(X_location_mix, y_location_mix, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_location_mix)
pipe_model_location_mix = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', random_state=RANDOM_STATE, class_weight='balanced'))
])
param_grid_location_mix = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['rbf', 'linear', 'poly']
}
model_grid_location_mix = GridSearchCV(
    estimator=pipe_model_location_mix, 
    param_grid=param_grid_location_mix, 
    scoring='f1_macro', 
    cv=5, 
    n_jobs=-1, 
    verbose=2
)

model_grid_location_mix.fit(Xtrain_location_mix, ytrain_location_mix)
model_location_mix = model_grid_location_mix.best_estimator_
print(f"Best parameters: {model_grid_location_mix.best_params_}")
ypred_location_mix = model_location_mix.predict(Xtest_location_mix)
f1_location_mix = f1_score(ytest_location_mix, ypred_location_mix, average='macro')
print(f"Mix: Location -> macro-F1: {f1_location_mix}")
print("Classification Report:")
print(classification_report(ytest_location_mix, ypred_location_mix))
print("Confusion Matrix:")
print(confusion_matrix(ytest_location_mix, ypred_location_mix))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'svc__C': 100, 'svc__gamma': 0.001, 'svc__kernel': 'linear'}
Mix: Location -> macro-F1: 0.08584239834239833
Classification Report:
              precision    recall  f1-score   support

        Apex       0.00      0.00      0.00         4
          LC       0.14      0.25      0.18         4
         LLA       0.40      0.50      0.44         4
        LLSB       0.11      0.25      0.15         4
         LMA       0.00      0.00      0.00         4
         LUA       0.00      0.00      0.00         3
        LUSB       0.00      0.00      0.00         3
          RC       0.00      0.00      0.00         4
         RLA       0.25      0.25      0.25         4
         RMA       0.00      0.00      0.00         3
         RUA       0.00      0.00      0.00         3
        RUSB       0.00      0.00      0.00         4

    accuracy                           0.11        44
   macro avg       0.08      0.

c:\Users\ardik\Anaconda3\envs\ExamMachineLearning\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ardik\Anaconda3\envs\ExamMachineLearning\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ardik\Anaconda3\envs\ExamMachineLearning\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

## LS Data

##### Gender

In [26]:
N_MFCC_GENDER_LS = 58
df_gender_ls = pd.read_csv(LS_CSV_PATH)
df_gender_ls['audio_path'] = df_gender_ls['Lung Sound ID'].apply(lambda x: normpath(join(LS_AUDIO_PATH, f"{str(x).strip()}.wav").replace('\\', '/')))
df_gender_ls['audio_path'] = df_gender_ls['audio_path'].str.replace('_C_', '_FC_').str.replace('_G_', '_CC_')
def extract_mfcc_gender_ls(filename, n_mfcc=N_MFCC_GENDER_LS):
    y, sr = librosa.load(filename, sr=44100)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    return np.concatenate([mfcc_mean, mfcc_std])

def flatten_mfcc(mfcc):
    return np.array(mfcc).flatten()

mfcc_list_gender_ls = []
labels_gender_ls = []   
for _, row in df_gender_ls.iterrows():
    try:
        mfcc_gender_ls = extract_mfcc_gender_ls(row['audio_path'])
        # mfcc_flat = flatten_mfcc(mfcc)
        mfcc_list_gender_ls.append(mfcc_gender_ls)
        labels_gender_ls.append(row['Gender'])
    except Exception as e:
        print(f"[WARN] Failed: {row['audio_path']} -> {e}")

X_gender_ls = pd.DataFrame(mfcc_list_gender_ls)
y_gender_ls = pd.Series(labels_gender_ls)

Xtrain_gender_ls, Xtest_gender_ls, ytrain_gender_ls, ytest_gender_ls = train_test_split(X_gender_ls, y_gender_ls, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_gender_ls)
pipe_model_gender_ls = Pipeline(steps=[
    ('scaler', StandardScaler()),   
    ('svc', SVC(kernel='rbf', random_state=RANDOM_STATE, class_weight='balanced'))
]) 

param_grid_gender_ls = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['rbf', 'linear', 'poly']
}
model_grid_gender_ls = GridSearchCV(
    estimator=pipe_model_gender_ls,
    param_grid=param_grid_gender_ls,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=2
)

model_grid_gender_ls.fit(Xtrain_gender_ls, ytrain_gender_ls)
model_gender_ls = model_grid_gender_ls.best_estimator_
print(f"Best parameters: {model_grid_gender_ls.best_params_}")
ypred_gender_ls = model_gender_ls.predict(Xtest_gender_ls)
f1_gender_ls = f1_score(ytest_gender_ls, ypred_gender_ls, average='macro')
print(f"LS: Gender -> macro-F1: {f1_gender_ls}")
print("Classification Report:")
print(classification_report(ytest_gender_ls, ypred_gender_ls))
print("Confusion Matrix:")
print(confusion_matrix(ytest_gender_ls, ypred_gender_ls))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'svc__C': 0.1, 'svc__gamma': 0.1, 'svc__kernel': 'poly'}
LS: Gender -> macro-F1: 0.5982142857142857
Classification Report:
              precision    recall  f1-score   support

           F       0.50      0.67      0.57         6
           M       0.71      0.56      0.62         9

    accuracy                           0.60        15
   macro avg       0.61      0.61      0.60        15
weighted avg       0.63      0.60      0.60        15

Confusion Matrix:
[[4 2]
 [4 5]]


##### Lung Sound type

In [27]:
N_MFCC_LUNGSOUNDTYPE_LS = 60
df_lungsoundtype_ls = pd.read_csv(LS_CSV_PATH)
df_lungsoundtype_ls['audio_path'] = df_lungsoundtype_ls['Lung Sound ID'].apply(lambda x: normpath(join(LS_AUDIO_PATH, f"{str(x).strip()}.wav").replace('\\', '/')))
df_lungsoundtype_ls['audio_path'] = df_lungsoundtype_ls['audio_path'].str.replace('_C_', '_FC_').str.replace('_G_', '_CC_')
def extract_mfcc_lungsoundtype_ls(filename, n_mfcc=N_MFCC_LUNGSOUNDTYPE_LS):
    y, sr = librosa.load(filename, sr=44100)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    return np.concatenate([mfcc_mean, mfcc_std])

mfcc_list_lungsoundtype_ls = []
labels_lungsoundtype_ls = []

for _, row in df_lungsoundtype_ls.iterrows():
    try:
        mfcc_lungsoundtype_ls = extract_mfcc_lungsoundtype_ls(row['audio_path'])
        # mfcc_flat = flatten_mfcc(mfcc)
        mfcc_list_lungsoundtype_ls.append(mfcc_lungsoundtype_ls)
        labels_lungsoundtype_ls.append(row['Lung Sound Type'])
    except Exception as e:
        print(f"[WARN] Failed: {row['audio_path']} -> {e}")

X_lungsoundtype_ls = pd.DataFrame(mfcc_list_lungsoundtype_ls)
y_lungsoundtype_ls = pd.Series(labels_lungsoundtype_ls)

Xtrain_lungsoundtype_ls, Xtest_lungsoundtype_ls, ytrain_lungsoundtype_ls, ytest_lungsoundtype_ls = train_test_split(X_lungsoundtype_ls, y_lungsoundtype_ls, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_lungsoundtype_ls)
pipe_model_lungsoundtype_ls = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', random_state=RANDOM_STATE, class_weight='balanced'))
])
param_grid_lungsoundtype_ls = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['rbf', 'linear', 'poly']
}
model_grid_lungsoundtype_ls = GridSearchCV(
    estimator=pipe_model_lungsoundtype_ls,
    param_grid=param_grid_lungsoundtype_ls,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=2
)
model_grid_lungsoundtype_ls.fit(Xtrain_lungsoundtype_ls, ytrain_lungsoundtype_ls)
model_lungsoundtype_ls = model_grid_lungsoundtype_ls.best_estimator_
print(f"Best parameters: {model_grid_lungsoundtype_ls.best_params_}")
ypred_lungsoundtype_ls = model_lungsoundtype_ls.predict(Xtest_lungsoundtype_ls)
f1_lungsoundtype_ls = f1_score(ytest_lungsoundtype_ls, ypred_lungsoundtype_ls, average='macro')
print(f"LS: Lung Sound Type -> macro-F1: {f1_lungsoundtype_ls}")
print("Classification Report:")
print(classification_report(ytest_lungsoundtype_ls, ypred_lungsoundtype_ls))
print("Confusion Matrix:")
print(confusion_matrix(ytest_lungsoundtype_ls, ypred_lungsoundtype_ls))

Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Users\ardik\Anaconda3\envs\ExamMachineLearning\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Best parameters: {'svc__C': 0.1, 'svc__gamma': 0.001, 'svc__kernel': 'linear'}
LS: Lung Sound Type -> macro-F1: 1.0
Classification Report:
                 precision    recall  f1-score   support

Coarse Crackles       1.00      1.00      1.00         3
  Fine Crackles       1.00      1.00      1.00         1
         Normal       1.00      1.00      1.00         4
    Pleural Rub       1.00      1.00      1.00         3
        Rhonchi       1.00      1.00      1.00         2
       Wheezing       1.00      1.00      1.00         2

       accuracy                           1.00        15
      macro avg       1.00      1.00      1.00        15
   weighted avg       1.00      1.00      1.00        15

Confusion Matrix:
[[3 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 4 0 0 0]
 [0 0 0 3 0 0]
 [0 0 0 0 2 0]
 [0 0 0 0 0 2]]


##### Location

In [28]:
N_MFCC_LOCATION_LS = 60
df_location_ls = pd.read_csv(LS_CSV_PATH)
df_location_ls['audio_path'] = df_location_ls['Lung Sound ID'].apply(lambda x: normpath(join(LS_AUDIO_PATH, f"{str(x).strip()}.wav").replace('\\', '/')))
df_location_ls['audio_path'] = df_location_ls['audio_path'].str.replace('_C_', '_FC_').str.replace('_G_', '_CC_')
def extract_mfcc_location_ls(filename, n_mfcc=N_MFCC_LOCATION_LS):
    y, sr = librosa.load(filename, sr=70000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    return np.concatenate([mfcc_mean, mfcc_std])
mfcc_list_location_ls = []
labels_location_ls = []
for _, row in df_location_ls.iterrows():
    try:
        mfcc_location_ls = extract_mfcc_location_ls(row['audio_path'])
        # mfcc_flat = flatten_mfcc(mfcc)
        mfcc_list_location_ls.append(mfcc_location_ls)
        labels_location_ls.append(row['Location'])
    except Exception as e:
        print(f"[WARN] Failed: {row['audio_path']} -> {e}")

X_location_ls = pd.DataFrame(mfcc_list_location_ls)
y_location_ls = pd.Series(labels_location_ls)
Xtrain_location_ls, Xtest_location_ls, ytrain_location_ls, ytest_location_ls = train_test_split(X_location_ls, y_location_ls, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_location_ls)
pipe_model_location_ls = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', random_state=RANDOM_STATE, class_weight='balanced'))
])
param_grid_location_ls = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['rbf', 'linear', 'poly']
}
model_grid_location_ls = GridSearchCV(
    estimator=pipe_model_location_ls,
    param_grid=param_grid_location_ls,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=2
)
model_grid_location_ls.fit(Xtrain_location_ls, ytrain_location_ls)
model_location_ls = model_grid_location_ls.best_estimator_
print(f"Best parameters: {model_grid_location_ls.best_params_}")
ypred_location_ls = model_location_ls.predict(Xtest_location_ls)
f1_location_ls = f1_score(ytest_location_ls, ypred_location_ls, average='macro')
print(f"LS: Location -> macro-F1: {f1_location_ls}")
print("Classification Report:")
print(classification_report(ytest_location_ls, ypred_location_ls))
print("Confusion Matrix:")
print(confusion_matrix(ytest_location_ls, ypred_location_ls))

Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Users\ardik\Anaconda3\envs\ExamMachineLearning\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Best parameters: {'svc__C': 10, 'svc__gamma': 0.01, 'svc__kernel': 'poly'}
LS: Location -> macro-F1: 0.3674603174603175
Classification Report:
              precision    recall  f1-score   support

         LLA       0.25      0.50      0.33         2
         LMA       0.50      0.67      0.57         3
         LUA       1.00      0.67      0.80         3
         RLA       1.00      0.33      0.50         3
         RMA       0.00      0.00      0.00         2
         RUA       0.00      0.00      0.00         2

    accuracy                           0.40        15
   macro avg       0.46      0.36      0.37        15
weighted avg       0.53      0.40      0.42        15

Confusion Matrix:
[[1 0 0 0 0 1]
 [0 2 0 0 1 0]
 [1 0 2 0 0 0]
 [2 0 0 1 0 0]
 [0 1 0 0 0 1]
 [0 1 0 0 1 0]]


Sumber AI : 
https://github.com/copilot/share/80665316-4184-8822-8812-a447a02f48e9
https://chatgpt.com/share/696cfe0c-cf00-800e-9ce4-f71726a7d39b